In [4]:
#!pip install kaggle

## Imports e Especificações de Libs

In [19]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

## EDA

In [3]:
path_to_data = r"data/"

In [23]:
user_review = pd.read_csv(f'{path_to_data}animelist.csv')
anime_list = pd.read_csv(f'{path_to_data}anime.csv')
anime_description = pd.read_csv(f'{path_to_data}anime_with_synopsis.csv')

In [24]:
#How many users
len(user_review.user_id.unique())

325770

In [25]:
#Showing first 10 anime ratings
for index, row in user_review.iterrows():
    print(index,row)
    if(index > 10):
        break

0 user_id              0
anime_id            67
rating               9
watching_status      1
watched_episodes     1
Name: 0, dtype: int64
1 user_id                0
anime_id            6702
rating                 7
watching_status        1
watched_episodes       4
Name: 1, dtype: int64
2 user_id               0
anime_id            242
rating               10
watching_status       1
watched_episodes      4
Name: 2, dtype: int64
3 user_id                0
anime_id            4898
rating                 0
watching_status        1
watched_episodes       1
Name: 3, dtype: int64
4 user_id              0
anime_id            21
rating              10
watching_status      1
watched_episodes     0
Name: 4, dtype: int64
5 user_id              0
anime_id            24
rating               9
watching_status      1
watched_episodes     5
Name: 5, dtype: int64
6 user_id                0
anime_id            2104
rating                 0
watching_status        1
watched_episodes       4
Name: 6, dtype

In [22]:
anime_list.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [43]:
anime_full = pd.merge(anime_list, anime_description[["MAL_ID","sypnopsis"]], how='inner', on="MAL_ID")

In [44]:
anime_description.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'sypnopsis'], dtype='object')

Missing Data

In [46]:
anime_full.isnull().sum().tail(5)

Score-4      0
Score-3      0
Score-2      0
Score-1      0
sypnopsis    8
dtype: int64

Choosing Features

In [60]:
anime_full.sample(1)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,sypnopsis
9438,30364,Haikyuu!! Movie 2: Shousha to Haisha,7.94,"Comedy, Sports, Drama, School, Shounen",Unknown,ハイキュー!! 勝者と敗者,Movie,1,"Sep 18, 2015",Unknown,...,2668.0,4893.0,3697.0,1186.0,496.0,134.0,26.0,16.0,207.0,Second Haikyuu!! recap movie.


In [48]:
features = ["Name", "Genres", "Episodes", "sypnopsis"]

SyntaxError: invalid syntax (1657071166.py, line 1)